In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


URL = 'https://finviz.com/quote.ashx?t='
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


def get_html(url, params=None):
  return requests.get(url, headers=HEADERS, params=params)


def get_industry(html):
  soup = BeautifulSoup(html, 'html.parser')
  data = soup.find_all('td', class_='fullview-links')

  soup = BeautifulSoup(str(data[1]), 'html.parser')
  data = soup.find_all('a')

  inry = list()
  for el in data:
    inry.append(el.contents[0])

  return inry[0], inry[1]


def parse(tic):
  html = get_html(URL + tic + '&ty=c&ta=1&p=d')
  if html.status_code == 200:
    print(tic)
    sectr, inry = get_industry(html.text)
    print(sectr, inry)
    return tic, sectr, inry
  else:
    print(tic)
    print('Error')
    return tic, 'NaN', 'NaN'


df = pd.read_csv('/spb_stocks.csv')
ticker = list(df['ticker'])

data = []
for tic in ticker:
  data.append(parse(tic))

print(data)

In [0]:
df_inry = pd.DataFrame(data, columns=['ticker', 'sector', 'industry'])

In [0]:
df_res = pd.merge(left=df, right=df_inry, on='ticker')
df_res.to_csv('/industry.csv', index=False)